### Conectando com o MongoDB e Carregando os Dados

In [ ]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

#Conectando no MongoDB - Sataging
client = MongoClient('mongodb://localhost:27017/')
db = client['IndustrialDataLake']
collection = db['MachineSensorData']

#Carregando os dados
df = pd.read_csv('dados/manufacturing_6G_dataset.csv')

#Criando um dicionário e convertendo nesse formato para inserir os dados no MongoDB
data = df.to_dict('records')
collection.insert_many(data)

print(f"Dados inseridos com sucesso! Total: {collection.count_documents({})} registros.")

Dados inseridos com sucesso! Total: 300000 registros.


### Criação das colunas do DW - SQL Server

In [ ]:
import pyodbc
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

#Configurando a conexão
SQL_SERVER = 'DESKTOP-U623P07'  
SQL_DATABASE = 'FactoryRepair'


#Realizando a conexão com o SQL Server
def create_sql_connection():
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SQL_SERVER};DATABASE={SQL_DATABASE};trusted_connection=yes;'
    return pyodbc.connect(conn_str)

#Criando a função para criar as tabelas no - DW SQL Server
def create_dw_tables():
    try:
        conn = create_sql_connection()
        cursor = conn.cursor()
        
        #Criando a tabela DimMachine 
        cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'DimMachine')
        CREATE TABLE DimMachine (
            MachineKey INT IDENTITY(1,1) PRIMARY KEY,
            MachineID INT NOT NULL,
            LoadDate DATETIME DEFAULT GETDATE(),
            CONSTRAINT UQ_MachineID UNIQUE (MachineID)
        );
        """)
        
        #Criando a tabela DimTime 
        cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'DimTime')
        CREATE TABLE DimTime (
            TimeKey INT IDENTITY(1,1) PRIMARY KEY,
            Timestamp DATETIME NOT NULL,
            Hour INT,
            Minute INT,
            LoadDate DATETIME DEFAULT GETDATE(),
            CONSTRAINT UQ_Timestamp UNIQUE (Timestamp)
        );
        """)
        
        #Criando a tabela FactProduction 
        cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'FactProduction')
        CREATE TABLE FactProduction (
            ProductionKey INT IDENTITY(1,1) PRIMARY KEY,
            MachineKey INT NOT NULL FOREIGN KEY REFERENCES DimMachine(MachineKey),
            TimeKey INT NOT NULL FOREIGN KEY REFERENCES DimTime(TimeKey),
            Temperature_C FLOAT,
            Vibration_Hz FLOAT,
            Power_Consumption_kW FLOAT,
            Production_Speed_units_per_hr FLOAT,
            Error_Rate_Percent FLOAT,
            Efficiency_Status VARCHAR(10),
            Operation_Mode VARCHAR(20),
            LoadDate DATETIME DEFAULT GETDATE(),
            CONSTRAINT UQ_MachineTime UNIQUE (MachineKey, TimeKey)
        );
        """)
        
        #Criando a tabela FactQuality 
        cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'FactQuality')
        CREATE TABLE FactQuality (
            QualityKey INT IDENTITY(1,1) PRIMARY KEY,
            MachineKey INT NOT NULL FOREIGN KEY REFERENCES DimMachine(MachineKey),
            TimeKey INT NOT NULL FOREIGN KEY REFERENCES DimTime(TimeKey),
            Quality_Control_Defect_Rate_Percent FLOAT,
            Network_Latency_ms FLOAT,
            Packet_Loss_Percent FLOAT,
            Predictive_Maintenance_Score FLOAT,
            LoadDate DATETIME DEFAULT GETDATE(),
            CONSTRAINT UQ_MachineTimeQuality UNIQUE (MachineKey, TimeKey)
        );
        """)
        
        conn.commit()
        print("Tabelas criadas com sucesso no Data Warehouse!")
        
    except Exception as e:
        print(f"Erro ao criar tabelas: {str(e)}")
    finally:
        if 'conn' in locals():
            conn.close()

#Executando a função de criação das tabelas acima
create_dw_tables()

Tabelas criadas com sucesso no Data Warehouse!


### Pipeline de ETL

In [ ]:
import pyodbc
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import time

#Conectando novamente ao SQL Server
SQL_SERVER = 'DESKTOP-U623P07'  
SQL_DATABASE = 'FactoryRepair'

#Criando uma função para criar a conexão com tratamento de erros
def create_sql_connection(max_retries=3, retry_delay=5):
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SQL_SERVER};DATABASE={SQL_DATABASE};trusted_connection=yes;'
    
    for attempt in range(max_retries):
        try:
            conn = pyodbc.connect(conn_str, timeout=30)
            print("Conexão com SQL Server estabelecida com sucesso!")
            return conn
        except pyodbc.Error as e:
            print(f"Tentativa {attempt + 1} de {max_retries} - Erro ao conectar ao SQL Server: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Aguardando {retry_delay} segundos antes de tentar novamente...")
                time.sleep(retry_delay)
    
    raise Exception("Não foi possível estabelecer conexão com o SQL Server após várias tentativas")

def load_to_sql_dw(df):
    conn = None
    cursor = None
    
    try:
        #Estabelecendo a conexão
        conn = create_sql_connection()
        cursor = conn.cursor()
        
        print("Iniciando processo de carga no Data Warehouse...")
        
        #Convertendo os tipos de dados do DataFrame
        print("Convertendo tipos de dados...")
        df['Machine_ID'] = df['Machine_ID'].astype(int)  # Converter para int nativo do Python
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        
        #Carregando a dimensão Máquina 
        print("Carregando dimensão Máquina...")
        machines = df[['Machine_ID']].drop_duplicates()
        machine_keys = {}
        
        for _, row in machines.iterrows():
            try:
                machine_id = int(row['Machine_ID'])  
                
                #Verificando se a máquina já existe
                cursor.execute("SELECT MachineKey FROM DimMachine WHERE MachineID = ?", machine_id)
                result = cursor.fetchone()
                
                if result:
                    machine_keys[machine_id] = result[0]
                else:
                    #Inserindo a nova máquina
                    cursor.execute("""
                    INSERT INTO DimMachine (MachineID) OUTPUT INSERTED.MachineKey VALUES (?)
                    """, machine_id)
                    machine_keys[machine_id] = cursor.fetchone()[0]
                    
            except Exception as e:
                print(f"Erro ao processar máquina ID {row['Machine_ID']}: {str(e)}")
                continue
        
        conn.commit()
        print(f"Dimensão Máquina carregada. Total: {len(machine_keys)} máquinas.")
        
        #Carregando a dimensão Tempo
        print("Carregando dimensão Tempo...")
        times = df[['Timestamp']].drop_duplicates()
        time_keys = {}
        
        for _, row in times.iterrows():
            try:
                timestamp = row['Timestamp'].to_pydatetime()  #Convertendo para datetime 
                hour = timestamp.hour
                minute = timestamp.minute
                
                #Verificando se o timestamp já existe
                cursor.execute("SELECT TimeKey FROM DimTime WHERE Timestamp = ?", timestamp)
                result = cursor.fetchone()
                
                if result:
                    time_keys[timestamp] = result[0]
                else:
                    #Inserindo o novo registro de tempo
                    cursor.execute("""
                    INSERT INTO DimTime (Timestamp, Hour, Minute) 
                    OUTPUT INSERTED.TimeKey 
                    VALUES (?, ?, ?)
                    """, timestamp, hour, minute)
                    time_keys[timestamp] = cursor.fetchone()[0]
                    
            except Exception as e:
                print(f"Erro ao processar timestamp {row['Timestamp']}: {str(e)}")
                continue
        
        conn.commit()
        print(f"Dimensão Tempo carregada. Total: {len(time_keys)} registros temporais.")
        
        #Carregando a fato Produção em lotes 
        print("Carregando fato Produção...")
        batch_size = 1000
        total_rows = len(df)
        inserted_rows = 0
        
        for i in range(0, total_rows, batch_size):
            batch = df.iloc[i:i + batch_size]
            batch_values = []
            
            for _, row in batch.iterrows():
                try:
                    machine_id = int(row['Machine_ID'])
                    timestamp = row['Timestamp'].to_pydatetime()
                    
                    machine_key = machine_keys.get(machine_id)
                    time_key = time_keys.get(timestamp)
                    
                    if machine_key is None or time_key is None:
                        continue
                    
                    #Convertendo todos os valores 
                    batch_values.append((
                        int(machine_key),
                        int(time_key),
                        float(row['Temperature_C']),
                        float(row['Vibration_Hz']),
                        float(row['Power_Consumption_kW']),
                        float(row['Production_Speed_units_per_hr']),
                        float(row['Error_Rate_%']),
                        str(row['Efficiency_Status']),
                        str(row['Operation_Mode'])
                    ))
                except Exception as e:
                    print(f"Erro ao preparar linha {i}: {str(e)}")
                    continue
            
            if batch_values:
                try:
                    cursor.executemany("""
                    INSERT INTO FactProduction (
                        MachineKey, TimeKey, Temperature_C, Vibration_Hz, 
                        Power_Consumption_kW, Production_Speed_units_per_hr, 
                        Error_Rate_Percent, Efficiency_Status, Operation_Mode
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """, batch_values)
                    inserted_rows += len(batch_values)
                    conn.commit()
                except Exception as e:
                    conn.rollback()
                    print(f"Erro ao inserir lote {i//batch_size}: {str(e)}")
        
        print(f"Fato Produção carregado. Total: {inserted_rows} registros inseridos.")
        
        #Carregando a fato Qualidade em lotes 
        print("Carregando fato Qualidade...")
        inserted_rows = 0
        
        for i in range(0, total_rows, batch_size):
            batch = df.iloc[i:i + batch_size]
            batch_values = []
            
            for _, row in batch.iterrows():
                try:
                    machine_id = int(row['Machine_ID'])
                    timestamp = row['Timestamp'].to_pydatetime()
                    
                    machine_key = machine_keys.get(machine_id)
                    time_key = time_keys.get(timestamp)
                    
                    if machine_key is None or time_key is None:
                        continue
                    
                    #Convertendo todos os valores
                    batch_values.append((
                        int(machine_key),
                        int(time_key),
                        float(row['Quality_Control_Defect_Rate_%']),
                        float(row['Network_Latency_ms']),
                        float(row['Packet_Loss_%']),
                        float(row['Predictive_Maintenance_Score'])
                    ))
                except Exception as e:
                    print(f"Erro ao preparar linha {i}: {str(e)}")
                    continue
            
            if batch_values:
                try:
                    cursor.executemany("""
                    INSERT INTO FactQuality (
                        MachineKey, TimeKey, Quality_Control_Defect_Rate_Percent,
                        Network_Latency_ms, Packet_Loss_Percent, Predictive_Maintenance_Score
                    ) VALUES (?, ?, ?, ?, ?, ?)
                    """, batch_values)
                    inserted_rows += len(batch_values)
                    conn.commit()
                except Exception as e:
                    conn.rollback()
                    print(f"Erro ao inserir lote {i//batch_size}: {str(e)}")
        
        print(f"Fato Qualidade carregado. Total: {inserted_rows} registros inseridos.")
        
        print("Carga no Data Warehouse concluída com sucesso!")
        
    except Exception as e:
        if conn:
            conn.rollback()
        print(f"Erro fatal durante a carga: {str(e)}")
        raise
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


def run_etl_pipeline():
    print("\n=== Iniciando pipeline ETL melhorado ===")
    
    try:
        #Extração dos dados
        print("\n[ETAPA 1] Extração dos dados do MongoDB")
        client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        db = client['IndustrialDataLake']
        collection = db['MachineSensorData']
        
        #Testando a conexão com o MongoDB
        client.server_info()
        
        #Extraindo do MongoDB
        print("Executando query no MongoDB...")
        data = list(collection.find({}, {'_id': 0}))
        
        if not data:
            raise Exception("Nenhum dado encontrado no MongoDB")
        
        df = pd.DataFrame(data)
        print(f"Extraídos {len(df)} registros do MongoDB")
        
        #Transformando os Dados
        print("\n[ETAPA 2] Transformação dos dados")
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Hour'] = df['Timestamp'].dt.hour
        df['Minute'] = df['Timestamp'].dt.minute
        
        #Verificando os dados faltantes
        print("Verificando dados faltantes...")
        missing_data = df.isnull().sum()
        if missing_data.any():
            print("Aviso: Dados faltantes encontrados:")
            print(missing_data[missing_data > 0])
            # Preencher ou remover valores nulos conforme necessário
            df.fillna(0, inplace=True)
        
        #Realizando a carga
        print("\n[ETAPA 3] Carga no SQL Server DW")
        load_to_sql_dw(df)
        
        print("\n=== Pipeline ETL concluído com sucesso! ===")
        
    except Exception as e:
        print(f"\n!!! Falha no pipeline ETL: {str(e)}")
    finally:
        if 'client' in locals():
            client.close()

#Executando todo o pipeline de ETL completo
if __name__ == "__main__":
    run_etl_pipeline()


=== Iniciando pipeline ETL melhorado ===

[ETAPA 1] Extração dos dados do MongoDB
Executando query no MongoDB...
Extraídos 300000 registros do MongoDB

[ETAPA 2] Transformação dos dados
Verificando dados faltantes...

[ETAPA 3] Carga no SQL Server DW
Conexão com SQL Server estabelecida com sucesso!
Iniciando processo de carga no Data Warehouse...
Convertendo tipos de dados...
Carregando dimensão Máquina...
Dimensão Máquina carregada. Total: 50 máquinas.
Carregando dimensão Tempo...
Dimensão Tempo carregada. Total: 100000 registros temporais.
Carregando fato Produção...
Erro ao inserir lote 0: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violação da restrição UNIQUE KEY 'UQ_MachineTime'. Não é possível inserir a chave duplicada no objeto 'dbo.FactProduction'. O valor de chave duplicada é (1, 1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]A instrução foi finalizada. (3621)")
Erro ao inserir lote 1: ('23000', "[23000

### Querys - Análises SQL

#### Eficiência por Máquina

In [ ]:
conn = create_sql_connection()
cursor = conn.cursor()

#Executando a consulta
query = '''
SELECT 
    m.MachineID,
    AVG(p.Production_Speed_units_per_hr) AS VelocidadeMediaProducao,
    AVG(p.Error_Rate_Percent) AS TaxaMediaErros,
    p.Efficiency_Status,
    COUNT(*) AS ContagemLeituras
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
GROUP BY m.MachineID, p.Efficiency_Status
ORDER BY VelocidadeMediaProducao DESC;
'''

df = pd.read_sql_query(query, conn)

#Formatando as colunas 
df['VelocidadeMediaProducao'] = df['VelocidadeMediaProducao'].apply(lambda x: f'R${x:,.2f}')
df['TaxaMediaErros'] = df['TaxaMediaErros'].apply(lambda x: f'{x:,.2f}%')

print(df)

Conexão com SQL Server estabelecida com sucesso!
     MachineID VelocidadeMediaProducao TaxaMediaErros Efficiency_Status  \
0            4                R$458.86          1.13%              High   
1           34                R$458.67          1.00%              High   
2           10                R$457.31          1.08%              High   
3           32                R$455.88          1.06%              High   
4           47                R$455.70          1.08%              High   
..         ...                     ...            ...               ...   
145         40                R$250.17          9.03%               Low   
146         17                R$250.13          8.89%               Low   
147         38                R$249.99          9.08%               Low   
148         46                R$249.20          8.73%               Low   
149          5                R$248.55          8.88%               Low   

     ContagemLeituras  
0                  51  
1 

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2172867650.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### OEE Eficácia Geral do Equipamento

In [ ]:
query = '''
SELECT 
    m.MachineID,
    AVG(p.Production_Speed_units_per_hr) / MAX(p.Production_Speed_units_per_hr) AS Disponibilidade,
    1 - AVG(q.Quality_Control_Defect_Rate_Percent / 100) AS Qualidade,
    (AVG(p.Production_Speed_units_per_hr) / MAX(p.Production_Speed_units_per_hr)) * 
    (1 - AVG(q.Quality_Control_Defect_Rate_Percent / 100)) AS OEE
FROM FactProduction p
JOIN FactQuality q ON p.MachineKey = q.MachineKey AND p.TimeKey = q.TimeKey
JOIN DimMachine m ON p.MachineKey = m.MachineKey
GROUP BY m.MachineID
ORDER BY OEE DESC;
'''

df = pd.read_sql_query(query, conn)

#Formatando as colunas 
df['Disponibilidade'] = df['Disponibilidade'].apply(lambda x: f'{x:,.2f}')
df['Qualidade'] = df['Qualidade'].apply(lambda x: f'{x*100:,.2f}%')
df['OEE'] = df['OEE'].apply(lambda x: f'{x:,.2f}')

print(df)

    MachineID Disponibilidade Qualidade   OEE
0          27            0.57    95.11%  0.54
1          16            0.57    94.98%  0.54
2          31            0.56    94.92%  0.54
3          23            0.56    94.99%  0.53
4          15            0.56    94.83%  0.53
5          39            0.56    94.93%  0.53
6          43            0.56    95.01%  0.53
7          26            0.56    95.06%  0.53
8           3            0.56    94.95%  0.53
9          24            0.56    95.01%  0.53
10         34            0.56    94.94%  0.53
11         44            0.56    95.02%  0.53
12          1            0.56    94.96%  0.53
13         22            0.56    94.90%  0.53
14         32            0.56    95.01%  0.53
15         37            0.55    95.11%  0.53
16         41            0.55    95.06%  0.53
17         45            0.55    95.16%  0.53
18         19            0.55    95.03%  0.53
19         20            0.55    94.97%  0.53
20         10            0.55    9

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2113942319.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


### Análise de Temperatura vs Defeitos

In [ ]:
query = '''
WITH TempRanges AS (
    SELECT
        m.MachineID,
        CASE
            WHEN p.Temperature_C < 50 THEN 'Baixa (<50)'
            WHEN p.Temperature_C BETWEEN 50 AND 70 THEN 'Normal (50-70)'
            WHEN p.Temperature_C BETWEEN 70 AND 85 THEN 'Alta (70-85)'
            ELSE 'Muito Alta (>85)'
        END AS FaixaTemperatura,
        q.Quality_Control_Defect_Rate_Percent AS DefectRate
    FROM FactProduction p
    JOIN FactQuality q ON p.MachineKey = q.MachineKey AND p.TimeKey = q.TimeKey
    JOIN DimMachine m ON p.MachineKey = m.MachineKey
    JOIN DimTime t ON p.TimeKey = t.TimeKey
)
SELECT
    FaixaTemperatura,
    AVG(DefectRate) AS TaxaDefeitoMedia,
    COUNT(*) AS TotalRegistros
FROM TempRanges
GROUP BY FaixaTemperatura
ORDER BY TaxaDefeitoMedia DESC;
'''

df = pd.read_sql_query(query, conn)

#Formatando as colunas 
df['TaxaDefeitoMedia'] = df['TaxaDefeitoMedia'].apply(lambda x: f'{x:,.2f}%')

print(df)

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2021189252.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


   FaixaTemperatura TaxaDefeitoMedia  TotalRegistros
0    Normal (50-70)            5.02%           33518
1       Baixa (<50)            5.01%           33198
2      Alta (70-85)            5.01%           24814
3  Muito Alta (>85)            4.97%            8470


### Análise de Desempenho por Turno / Horário

In [ ]:
#Configurando a conexão no SQL Novamente
SQL_SERVER = 'DESKTOP-U623P07' 
SQL_DATABASE = 'FactoryRepair'

#Função para criar conexão com o tratamento caso der erro de conexão
def create_sql_connection(max_retries=3, retry_delay=5):
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SQL_SERVER};DATABASE={SQL_DATABASE};trusted_connection=yes;'
    
    for attempt in range(max_retries):
        try:
            conn = pyodbc.connect(conn_str, timeout=30)
            print("Conexão com SQL Server estabelecida com sucesso!")
            return conn
        except pyodbc.Error as e:
            print(f"Tentativa {attempt + 1} de {max_retries} - Erro ao conectar ao SQL Server: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Aguardando {retry_delay} segundos antes de tentar novamente...")
                time.sleep(retry_delay)
    
    raise Exception("Não foi possível estabelecer conexão com o SQL Server após várias tentativas")

def load_to_sql_dw(df):
    conn = None
    cursor = None

conn = create_sql_connection()
cursor = conn.cursor()

query = '''
SELECT 
    DATEPART(HOUR, t.Timestamp) AS Hora,
    FORMAT(AVG(p.Production_Speed_units_per_hr), 'N2') AS VelocidadeMediaProducao,
    CONCAT(FORMAT(AVG(p.Error_Rate_Percent), 'N2'), '%') AS TaxaErroMedia,
    COUNT(*) AS TotalRegistros
FROM FactProduction p
JOIN DimTime t ON p.TimeKey = t.TimeKey
GROUP BY DATEPART(HOUR, t.Timestamp)
ORDER BY Hora;
'''

df = pd.read_sql_query(query, conn)

print(df)

Conexão com SQL Server estabelecida com sucesso!
    Hora VelocidadeMediaProducao TaxaErroMedia  TotalRegistros
0      0                  275,51         7,37%            4200
1      1                  274,89         7,63%            4200
2      2                  278,73         7,50%            4200
3      3                  277,80         7,46%            4200
4      4                  275,03         7,46%            4200
5      5                  274,59         7,47%            4200
6      6                  277,93         7,47%            4200
7      7                  276,18         7,61%            4200
8      8                  276,30         7,39%            4200
9      9                  275,23         7,49%            4200
10    10                  273,41         7,43%            4180
11    11                  273,87         7,28%            4140
12    12                  278,19         7,60%            4140
13    13                  274,12         7,57%            4140
14    

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\3887636270.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Comparação Entre Máquinas Ativas e Ociosas

In [47]:
query = '''
SELECT 
    CASE 
        WHEN p.Operation_Mode = 'Active' THEN 'Ativa'
        ELSE 'Ociosa/Manutenção'
    END AS StatusOperacao,
    COUNT(*) AS TotalRegistros,
    FORMAT(AVG(p.Temperature_C), 'N2') AS TemperaturaMedia,
    FORMAT(AVG(p.Vibration_Hz), 'N2') AS VibracaoMedia,
    FORMAT(AVG(p.Power_Consumption_kW), 'N2') AS ConsumoEnergiaMedio
FROM FactProduction p
GROUP BY CASE 
        WHEN p.Operation_Mode = 'Active' THEN 'Ativa'
        ELSE 'Ociosa/Manutenção'
    END;
'''

df = pd.read_sql_query(query, conn)

print(df)

      StatusOperacao  TotalRegistros TemperaturaMedia VibracaoMedia  \
0              Ativa           70054            60,06          2,55   
1  Ociosa/Manutenção           29946            60,00          2,54   

  ConsumoEnergiaMedio  
0                5,75  
1                5,73  


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\3683968271.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Máquinas com maior taxa de defeitos

In [52]:
query = '''
SELECT TOP 10
    m.MachineID,
    CONCAT(FORMAT(AVG(q.Quality_Control_Defect_Rate_Percent), 'N2'), '%') AS TaxaDefeitoMedia,
    COUNT(*) AS TotalRegistros
FROM FactQuality q
JOIN DimMachine m ON q.MachineKey = m.MachineKey
GROUP BY m.MachineID
ORDER BY TaxaDefeitoMedia DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

   MachineID TaxaDefeitoMedia  TotalRegistros
0         15            5,17%            2027
1          9            5,15%            1991
2         14            5,11%            1941
3         22            5,10%            2031
4         11            5,09%            1951
5         47            5,09%            2001
6         31            5,08%            1933
7         39            5,07%            1993
8          8            5,07%            1967
9         36            5,07%            2085


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2288380342.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Correlação entre temperatura e defeitos

In [ ]:
query = '''
WITH TempRanges AS (
    SELECT
        m.MachineID,
        CASE
            WHEN p.Temperature_C < 50 THEN 'Baixa (<50)'
            WHEN p.Temperature_C BETWEEN 50 AND 70 THEN 'Normal (50-70)'
            WHEN p.Temperature_C BETWEEN 70 AND 85 THEN 'Alta (70-85)'
            ELSE 'Muito Alta (>85)'
        END AS FaixaTemperatura,
        q.Quality_Control_Defect_Rate_Percent
    FROM FactProduction p
    JOIN FactQuality q ON p.MachineKey = q.MachineKey AND p.TimeKey = q.TimeKey
    JOIN DimMachine m ON p.MachineKey = m.MachineKey
)
SELECT
    FaixaTemperatura,
    CONCAT(FORMAT(AVG(Quality_Control_Defect_Rate_Percent), 'N2'), '%') AS TaxaDefeitoMedia,
    COUNT(*) AS TotalRegistros
FROM TempRanges
GROUP BY FaixaTemperatura
ORDER BY 
    CASE FaixaTemperatura
        WHEN 'Baixa (<50)' THEN 1
        WHEN 'Normal (50-70)' THEN 2
        WHEN 'Alta (70-85)' THEN 3
        ELSE 4
    END;
'''

df = pd.read_sql_query(query, conn)

print(df)

   FaixaTemperatura TaxaDefeitoMedia  TotalRegistros
0       Baixa (<50)            5,01%           33198
1    Normal (50-70)            5,02%           33518
2      Alta (70-85)            5,01%           24814
3  Muito Alta (>85)            4,97%            8470


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2997150497.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Consumo de energia por modo de operação

In [ ]:
query = '''
SELECT
    Operation_Mode AS ModoOperacao,
    FORMAT(AVG(Power_Consumption_kW), 'N2') AS ConsumoMedioEnergia,
    FORMAT(MIN(Power_Consumption_kW), 'N2') AS ConsumoMinimo,
    FORMAT(MAX(Power_Consumption_kW), 'N2') AS ConsumoMaximo,
    COUNT(*) AS TotalRegistros
FROM FactProduction
GROUP BY Operation_Mode
ORDER BY ConsumoMedioEnergia DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

  ModoOperacao ConsumoMedioEnergia ConsumoMinimo ConsumoMaximo  TotalRegistros
0       Active                5,75          1,50         10,00           70054
1         Idle                5,73          1,50         10,00           20057
2  Maintenance                5,73          1,50         10,00            9889


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\260483734.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Máquinas Menos Eficientes Energeticamente

In [ ]:
query = '''
SELECT TOP 10
    m.MachineID,
    FORMAT(AVG(p.Power_Consumption_kW / p.Production_Speed_units_per_hr), 'N2') AS ConsumoPorUnidade,
    FORMAT(AVG(p.Power_Consumption_kW), 'N2') AS ConsumoMedioEnergia,
    FORMAT(AVG(p.Production_Speed_units_per_hr), 'N2') AS VelocidadeMediaProducao
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
WHERE p.Operation_Mode = 'Active'
GROUP BY m.MachineID
ORDER BY ConsumoPorUnidade DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

   MachineID ConsumoPorUnidade ConsumoMedioEnergia VelocidadeMediaProducao
0         21              0,03                5,90                  271,96
1         26              0,03                5,74                  280,64
2         40              0,03                5,76                  271,68
3         16              0,03                5,82                  286,25
4          4              0,03                5,73                  271,12
5         15              0,03                5,81                  281,65
6          2              0,03                5,73                  275,21
7         11              0,03                5,75                  277,52
8          7              0,03                5,69                  274,96
9          1              0,03                5,83                  277,77


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\3816957555.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Identificar máquinas com vibração anormal

In [ ]:
query = '''
SELECT
    m.MachineID,
    FORMAT(AVG(p.Vibration_Hz), 'N2') AS VibracaoMedia,
    FORMAT(STDEV(p.Vibration_Hz), 'N2') AS DesvioPadraoVibracao,
    FORMAT(MAX(p.Vibration_Hz), 'N2') AS VibracaoMaxima,
    COUNT(*) AS TotalRegistros
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
WHERE p.Operation_Mode = 'Active'
GROUP BY m.MachineID
HAVING AVG(p.Vibration_Hz) > 3.5 OR STDEV(p.Vibration_Hz) > 1.2
ORDER BY VibracaoMedia DESC;
'''

df = pd.read_sql_query(query, conn)


print(df)

    MachineID VibracaoMedia DesvioPadraoVibracao VibracaoMaxima  \
0          34          2,66                 1,42           5,00   
1           3          2,63                 1,43           5,00   
2          19          2,60                 1,41           4,99   
3          37          2,60                 1,43           5,00   
4          26          2,59                 1,43           5,00   
5          38          2,59                 1,41           4,99   
6           1          2,58                 1,40           5,00   
7          15          2,58                 1,41           5,00   
8          36          2,58                 1,43           4,99   
9          50          2,57                 1,41           5,00   
10         23          2,57                 1,42           5,00   
11          4          2,57                 1,41           4,99   
12         45          2,57                 1,40           5,00   
13         47          2,57                 1,42           5,0

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\3417990692.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Padrões antes de falhas (análise de tendência)

In [ ]:
query = '''
WITH Falhas AS (
    SELECT 
        m.MachineID,
        t.Timestamp,
        p.Error_Rate_Percent,
        LAG(p.Temperature_C, 1) OVER (PARTITION BY m.MachineID ORDER BY t.Timestamp) AS TempAnterior,
        LAG(p.Vibration_Hz, 1) OVER (PARTITION BY m.MachineID ORDER BY t.Timestamp) AS VibAnterior,
        LAG(p.Power_Consumption_kW, 1) OVER (PARTITION BY m.MachineID ORDER BY t.Timestamp) AS PowerAnterior
    FROM FactProduction p
    JOIN DimMachine m ON p.MachineKey = m.MachineKey
    JOIN DimTime t ON p.TimeKey = t.TimeKey
    WHERE p.Error_Rate_Percent > 5  -- Considerando como "falha" quando ErrorRate > 5%
)
SELECT
    MachineID,
    FORMAT(AVG(TempAnterior), 'N2') AS TemperaturaMediaAntesFalha,
    FORMAT(AVG(VibAnterior), 'N2') AS VibracaoMediaAntesFalha,
    FORMAT(AVG(PowerAnterior), 'N2') AS ConsumoEnergiaMedioAntesFalha,
    COUNT(*) AS TotalFalhasAnalisadas
FROM Falhas
GROUP BY MachineID;
'''

df = pd.read_sql_query(query, conn)

print(df)

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\2184155748.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


    MachineID TemperaturaMediaAntesFalha VibracaoMediaAntesFalha  \
0           1                      59,49                    2,57   
1           3                      59,07                    2,61   
2          10                      60,31                    2,56   
3          12                      59,57                    2,50   
4          17                      60,17                    2,59   
5          19                      59,72                    2,55   
6          26                      60,08                    2,63   
7          28                      59,26                    2,55   
8          35                      60,27                    2,51   
9          37                      60,20                    2,55   
10         42                      60,56                    2,54   
11         44                      60,16                    2,58   
12          2                      59,91                    2,55   
13          4                      59,68        

#### Análise de Eficiência Geral (OEE)

In [59]:
query = '''
SELECT
    m.MachineID,
    -- Disponibilidade (Tempo Ativo / Tempo Total)
    FORMAT(CAST(SUM(CASE WHEN p.Operation_Mode = 'Active' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*), 'N2') AS Disponibilidade,
    
    -- Desempenho (Produção Real / Produção Teórica Máxima)
    FORMAT(AVG(p.Production_Speed_units_per_hr) / MAX(p.Production_Speed_units_per_hr), 'N2') AS Desempenho,
    
    -- Qualidade (1 - Taxa de Defeitos)
    FORMAT(1 - AVG(q.Quality_Control_Defect_Rate_Percent / 100), 'N2') AS Qualidade,
    
    -- OEE Total (Disponibilidade × Desempenho × Qualidade)
    FORMAT(
        (CAST(SUM(CASE WHEN p.Operation_Mode = 'Active' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*)) *
        (AVG(p.Production_Speed_units_per_hr) / MAX(p.Production_Speed_units_per_hr)) *
        (1 - AVG(q.Quality_Control_Defect_Rate_Percent / 100)),
        'N2'
    ) AS OEE,
    
    COUNT(*) AS TotalRegistros
FROM FactProduction p
JOIN FactQuality q ON p.MachineKey = q.MachineKey AND p.TimeKey = q.TimeKey
JOIN DimMachine m ON p.MachineKey = m.MachineKey
GROUP BY m.MachineID
ORDER BY OEE DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

    MachineID Disponibilidade Desempenho Qualidade   OEE  TotalRegistros
0          48            0,72       0,55      0,95  0,38            2058
1          15            0,71       0,56      0,95  0,38            2027
2          16            0,71       0,57      0,95  0,38            2010
3          31            0,70       0,56      0,95  0,38            1933
4          27            0,70       0,57      0,95  0,38            1957
5          23            0,71       0,56      0,95  0,38            2092
6          25            0,71       0,55      0,95  0,37            1998
7          49            0,71       0,55      0,95  0,37            1950
8           6            0,71       0,55      0,95  0,37            1934
9          34            0,69       0,56      0,95  0,37            1948
10         42            0,71       0,55      0,95  0,37            2019
11         45            0,70       0,55      0,95  0,37            2080
12          8            0,70       0,55      0,95 

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\573831378.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Análise Temporal (Tendências)

In [ ]:
query = '''
SELECT
    CAST(t.Timestamp AS DATE) AS Data,
    FORMAT(AVG(p.Production_Speed_units_per_hr), 'N2') AS VelocidadeMediaProducao,
    FORMAT(AVG(p.Error_Rate_Percent), 'N2') AS TaxaErroMedia,
    FORMAT(AVG(q.Quality_Control_Defect_Rate_Percent), 'N2') AS TaxaDefeitoMedia,
    COUNT(*) AS TotalRegistros
FROM FactProduction p
JOIN FactQuality q ON p.MachineKey = q.MachineKey AND p.TimeKey = q.TimeKey
JOIN DimTime t ON p.TimeKey = t.TimeKey
GROUP BY CAST(t.Timestamp AS DATE)
ORDER BY Data;
'''

df = pd.read_sql_query(query, conn)

print(df)

          Data VelocidadeMediaProducao TaxaErroMedia TaxaDefeitoMedia  \
0   2024-01-01                  273,83          7,53             5,02   
1   2024-01-02                  277,94          7,55             5,12   
2   2024-01-03                  276,06          7,42             5,05   
3   2024-01-04                  270,64          7,51             4,98   
4   2024-01-05                  282,10          7,37             5,04   
..         ...                     ...           ...              ...   
65  2024-03-06                  276,85          7,72             4,98   
66  2024-03-07                  280,23          7,38             4,98   
67  2024-03-08                  276,97          7,56             4,98   
68  2024-03-09                  279,33          7,55             5,08   
69  2024-03-10                  270,14          7,39             4,86   

    TotalRegistros  
0             1440  
1             1440  
2             1440  
3             1440  
4             1440

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\414578384.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Comparação dia da semana vs fim de semana

In [62]:
query = '''
SELECT
    CASE 
        WHEN DATEPART(WEEKDAY, t.Timestamp) IN (1, 7) THEN 'Fim de Semana'
        ELSE 'Dia de Semana'
    END AS TipoDia,
    FORMAT(AVG(p.Production_Speed_units_per_hr), 'N2') AS VelocidadeMediaProducao,
    CONCAT(FORMAT(AVG(p.Error_Rate_Percent), 'N2'), '%') AS TaxaErroMedia,
    FORMAT(AVG(p.Power_Consumption_kW), 'N2') AS ConsumoMedioEnergia,
    COUNT(*) AS TotalRegistros
FROM FactProduction p
JOIN DimTime t ON p.TimeKey = t.TimeKey
GROUP BY CASE 
        WHEN DATEPART(WEEKDAY, t.Timestamp) IN (1, 7) THEN 'Fim de Semana'
        ELSE 'Dia de Semana'
    END;
'''

df = pd.read_sql_query(query, conn)

print(df)

         TipoDia VelocidadeMediaProducao TaxaErroMedia ConsumoMedioEnergia  \
0  Dia de Semana                  276,29         7,51%                5,75   
1  Fim de Semana                  274,95         7,48%                5,74   

   TotalRegistros  
0           72000  
1           28000  


C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\1603125344.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Status atual das máquinas (última leitura de cada uma)

In [65]:
query = '''
WITH UltimasLeituras AS (
    SELECT
        p.MachineKey,
        p.Operation_Mode,
        p.Temperature_C,
        p.Vibration_Hz,
        p.Production_Speed_units_per_hr,
        p.Error_Rate_Percent,
        p.Efficiency_Status,
        t.Timestamp,
        ROW_NUMBER() OVER (PARTITION BY p.MachineKey ORDER BY t.Timestamp DESC) AS RN
    FROM FactProduction p
    JOIN DimTime t ON p.TimeKey = t.TimeKey
)
SELECT
    m.MachineID,
    ul.Operation_Mode AS StatusAtual,
    ul.Temperature_C AS TempAtual,
    ul.Vibration_Hz AS VibracaoAtual,
    ul.Production_Speed_units_per_hr AS VelocidadeProducaoAtual,
    CONCAT(FORMAT(ul.Error_Rate_Percent, 'N2'), '%') AS TaxaErroAtual,
    ul.Efficiency_Status AS StatusEficiencia,
    ul.Timestamp AS UltimaLeitura
FROM UltimasLeituras ul
JOIN DimMachine m ON ul.MachineKey = m.MachineKey
WHERE ul.RN = 1
ORDER BY m.MachineID;
'''

df = pd.read_sql_query(query, conn)

print(df)

    MachineID  StatusAtual  TempAtual  VibracaoAtual  VelocidadeProducaoAtual  \
0           1       Active  41.500657       4.075691               428.364469   
1           2         Idle  68.769712       4.954372               411.605937   
2           3         Idle  71.143556       1.861171               282.822791   
3           4       Active  61.238913       0.297255               398.884431   
4           5       Active  76.527323       1.827437               290.835686   
5           6       Active  89.658898       4.284868               293.205641   
6           7         Idle  51.038814       3.853015               419.646252   
7           8         Idle  76.895243       2.278493               120.726780   
8           9         Idle  54.569469       3.830994               321.824057   
9          10       Active  52.800333       2.629977               324.277200   
10         11       Active  67.812726       2.633789               462.782459   
11         12  Maintenance  

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\300804319.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Alertas de possíveis problemas

In [66]:
query = '''
SELECT
    m.MachineID,
    'Temperatura Alta' AS TipoAlerta,
    FORMAT(p.Temperature_C, 'N2') AS Valor,
    t.Timestamp
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
JOIN DimTime t ON p.TimeKey = t.TimeKey
WHERE p.Temperature_C > 85  -- Limite de temperatura
UNION ALL
SELECT
    m.MachineID,
    'Vibração Excessiva' AS TipoAlerta,
    FORMAT(p.Vibration_Hz, 'N2') AS Valor,
    t.Timestamp
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
JOIN DimTime t ON p.TimeKey = t.TimeKey
WHERE p.Vibration_Hz > 4.5  -- Limite de vibração
ORDER BY Timestamp DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\4103534769.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


       MachineID          TipoAlerta  Valor           Timestamp
0             47  Vibração Excessiva   4,84 2024-03-10 10:23:00
1             45    Temperatura Alta  86,48 2024-03-10 10:18:00
2             45  Vibração Excessiva   4,52 2024-03-10 10:18:00
3             24  Vibração Excessiva   4,96 2024-03-10 10:13:00
4             12  Vibração Excessiva   4,82 2024-03-10 10:12:00
...          ...                 ...    ...                 ...
18576         15  Vibração Excessiva   4,71 2024-01-01 00:34:00
18577         27  Vibração Excessiva   4,54 2024-01-01 00:30:00
18578         24    Temperatura Alta  89,33 2024-01-01 00:18:00
18579         40    Temperatura Alta  89,78 2024-01-01 00:13:00
18580         39  Vibração Excessiva   4,82 2024-01-01 00:06:00

[18581 rows x 4 columns]


#### Ranking de máquinas por produtividade

In [ ]:
query = '''
SELECT
    m.MachineID,
    FORMAT(AVG(p.Production_Speed_units_per_hr), 'N2') AS VelocidadeMediaProducao,
    RANK() OVER (ORDER BY AVG(p.Production_Speed_units_per_hr) DESC) AS RankProdutividade,
    CONCAT(FORMAT(AVG(p.Error_Rate_Percent), 'N2'), '%') AS TaxaErroMedia,
    RANK() OVER (ORDER BY AVG(p.Error_Rate_Percent)) AS RankQualidade,
    FORMAT(AVG(p.Power_Consumption_kW / p.Production_Speed_units_per_hr), 'N2') AS EficienciaEnergetica,
    RANK() OVER (ORDER BY AVG(p.Power_Consumption_kW / p.Production_Speed_units_per_hr)) AS RankEficienciaEnergetica
FROM FactProduction p
JOIN DimMachine m ON p.MachineKey = m.MachineKey
WHERE p.Operation_Mode = 'Active'
GROUP BY m.MachineID
ORDER BY RankProdutividade;
'''

df = pd.read_sql_query(query, conn)


print(df)

    MachineID VelocidadeMediaProducao  RankProdutividade TaxaErroMedia  \
0          27                  287,43                  1         7,45%   
1          16                  286,25                  2         7,41%   
2          34                  282,63                  3         7,53%   
3          31                  282,34                  4         7,56%   
4          39                  281,88                  5         7,59%   
5          15                  281,65                  6         7,40%   
6          43                  281,43                  7         7,45%   
7          26                  280,64                  8         7,42%   
8           3                  280,61                  9         7,62%   
9          23                  279,71                 10         7,41%   
10         32                  279,49                 11         7,53%   
11         24                  279,15                 12         7,40%   
12         42                  278,93 

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\3570476516.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Máquinas com comportamento Diferente (desvio padrão)

In [69]:
query = '''
WITH DesviosPorMaquina AS (
    SELECT
        m.MachineID,
        STDEV(p.Production_Speed_units_per_hr) AS DesvioPadraoProducao,
        STDEV(p.Temperature_C) AS DesvioPadraoTemperatura,
        STDEV(p.Vibration_Hz) AS DesvioPadraoVibracao
    FROM FactProduction p
    JOIN DimMachine m ON p.MachineKey = m.MachineKey
    WHERE p.Operation_Mode = 'Active'
    GROUP BY m.MachineID
),

MediasDesvios AS (
    SELECT
        AVG(DesvioPadraoProducao) AS MediaDesvioProducao,
        AVG(DesvioPadraoTemperatura) AS MediaDesvioTemperatura,
        AVG(DesvioPadraoVibracao) AS MediaDesvioVibracao
    FROM DesviosPorMaquina
)

SELECT
    d.MachineID,
    FORMAT(d.DesvioPadraoProducao, 'N2') AS DesvioPadraoProducao,
    FORMAT(d.DesvioPadraoTemperatura, 'N2') AS DesvioPadraoTemperatura,
    FORMAT(d.DesvioPadraoVibracao, 'N2') AS DesvioPadraoVibracao,
    FORMAT(m.MediaDesvioProducao, 'N2') AS MediaDesvioProducao,
    FORMAT(m.MediaDesvioTemperatura, 'N2') AS MediaDesvioTemperatura,
    FORMAT(m.MediaDesvioVibracao, 'N2') AS MediaDesvioVibracao
FROM DesviosPorMaquina d
CROSS JOIN MediasDesvios m
WHERE d.DesvioPadraoProducao > m.MediaDesvioProducao
   OR d.DesvioPadraoTemperatura > m.MediaDesvioTemperatura
   OR d.DesvioPadraoVibracao > m.MediaDesvioVibracao
ORDER BY d.DesvioPadraoProducao DESC;
'''

df = pd.read_sql_query(query, conn)

print(df)

    MachineID DesvioPadraoProducao DesvioPadraoTemperatura  \
0           8               133,41                   17,45   
1          46               132,79                   17,49   
2          12               132,40                   17,36   
3           5               132,35                   17,39   
4          34               132,04                   17,38   
5          32               131,75                   17,31   
6          47               131,72                   17,05   
7          48               131,54                   17,39   
8          35               131,12                   17,33   
9          42               131,12                   17,42   
10         44               131,10                   17,33   
11         26               131,09                   17,01   
12         27               130,77                   17,67   
13          7               130,70                   17,55   
14         45               130,64                   17,28   
15      

C:\Users\Matt\AppData\Local\Temp\ipykernel_14568\1857946295.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
